In [1]:
#Kreacija dataseta

%run ./gen_images.py

%run ./prep_dataset.py

dset = dataset.HiraSet('dset', 22500)
dset.pull()
(train_data,test_data,train_labels,test_labels) = dset.require_new(25,20)

/home/dradisavljevic/anaconda3/lib/python3.7/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


In [3]:
#CNN

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.utils import np_utils
from keras import backend as K
import datetime
K.set_image_dim_ordering('th')

print(datetime.datetime.now())
train_data=np.array(train_data)
test_data=np.array(test_data)
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)
train_data = train_data.reshape(train_data.shape[0], 1, 150, 150).astype('float32')
test_data = test_data.reshape(test_data.shape[0], 1, 150, 150).astype('float32')


def larger_model():
    # create model
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(1, 150, 150), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(74, activation='softmax'))
    # Compile model
    model.compile(loss='mse', optimizer='rmsprop', metrics=['mae', 'mape'])
    return model


# build the model
model = larger_model()
# Fit the model
model.fit(train_data, train_labels, epochs=300, batch_size=50, verbose=0)

scores = model.evaluate(test_data, test_labels, verbose=0)
print("Baseline Error: %.2f%%" % (scores[1]*100))
print(datetime.datetime.now())

# Accuracy : 98.74%
# Time of Execution : ~25m

2019-04-12 19:45:07.990797
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Baseline Error: 1.33%
2019-04-12 22:35:14.496736


In [39]:
# K Nearest Neighbours

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

nsamples, nx, ny, nz = train_data.shape
d2_train_data = train_data.reshape((nsamples,nx*ny*nz))

model = KNeighborsClassifier(n_neighbors = 1)
model.fit(d2_train_data, train_labels)

#printing out the score for test sample

nsamples, nx, ny, nz = test_data.shape
d2_test_data = test_data.reshape((nsamples,nx*ny*nz))
prediction = model.predict(d2_test_data)
acc_knn = accuracy_score(test_labels, prediction)
print('Nearest neighbours accuracy: ',acc_knn)

# Neighbour count = 1 Accuracy = 49.26%
# Neighbour count = 2 Accuracy = 23.65%
# Neighbour count = 3 Accuracy = 31.96%
# Neighbour count = 4 Accuracy = 17.97%
# Neighbour count = 5 Accuracy = 21.08%
# Neighbour count = 6 Accuracy = 11.82%
# Time of Execution : ~93s

AttributeError: 'list' object has no attribute 'shape'

In [11]:
# Long Short Term Memory Recurrent Neural Network

from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras import backend as K
K.set_image_dim_ordering('th')
import datetime

print(datetime.datetime.now())
train_data=np.array(train_data)
test_data=np.array(test_data)
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)
train_data = train_data.reshape(train_data.shape[0], 1, 150, 150).astype('float32')
test_data = test_data.reshape(test_data.shape[0], 1, 150, 150).astype('float32')

nsamples, nx, ny, nz = train_data.shape
d3_train_data = train_data.reshape((nsamples,nx,ny*nz))

model = Sequential()
model.add(LSTM(2, batch_input_shape=(10, 1, 22500), input_shape=(,22500), stateful=True, return_sequences=True))
model.add(LSTM(2, batch_input_shape=(10, 1, 22500), input_shape=(,22500), stateful=True))
model.add(Dense(74))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(d3_train_data, train_labels, epochs=300, batch_size=10, verbose=0)
print(datetime.datetime.now())

# Time of Execution : ~45m

2019-04-14 12:56:07.004445


/home/dradisavljevic/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/dradisavljevic/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(2, batch_input_shape=(10, 1, 22..., stateful=True, return_sequences=True, input_shape=(None, 225...)`
/home/dradisavljevic/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/dradisavljevic/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(2, batch_input_shape=(10, 1, 22..., stateful=True, input_shape=(None, 225...)`


2019-04-14 13:01:44.962938


In [12]:
#Predict LSTM

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import datetime

print(datetime.datetime.now())
scaler = MinMaxScaler(feature_range=(0, 1))
nsamples, nx, ny, nz = test_data.shape
d3_test_data = test_data.reshape((nsamples,nx,ny*nz))

testPredict = model.predict(d3_test_data, batch_size=10)
print(datetime.datetime.now())

2019-04-14 13:07:49.298487
2019-04-14 13:07:49.953115


In [13]:
#Test Score LSTM

print(datetime.datetime.now())
scores = model.evaluate(d3_test_data, test_labels, verbose=0, batch_size=10)
print("Baseline Error: %.2f%%" % (100-scores*100))
print(datetime.datetime.now())

# Accuracy : 1.33%

2019-04-14 13:07:52.563389
Baseline Error: 98.67%
2019-04-14 13:07:53.248309


In [18]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

(train_data,test_data,train_labels,test_labels) = dset.require_new(25,20)
train_data=np.array(train_data)
test_data=np.array(test_data)
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)

model = RandomForestClassifier(n_estimators=50)
model.fit(train_data, train_labels)
prediction = model.predict(test_data)
acc_rf = accuracy_score(test_labels, prediction)
print ("Random forest accuracy: ",acc_rf)

# Accuracy : 9.26%
# Time of Execution : ~15s

Random forest accuracy:  0.08581081081081081


In [23]:
# Stochastic Gradient Descent

from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier

(train_data,test_data,train_labels,test_labels) = dset.require_new_RNN(25,20)
train_data=np.array(train_data)
test_data=np.array(test_data)
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)


model = SGDClassifier(max_iter=100,tol=0.003)
model.fit(train_data, train_labels)
prediction = model.predict(test_data)
acc_sgd = accuracy_score(test_labels, prediction)
print ("Stochastic gradient descent accuracy: ",acc_sgd)

# Accuracy : 8.45%
# Time of Execution : ~30s

Stochastic gradient descent accuracy:  0.26216216216216215


In [24]:
# Linear Support Vector Machine

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

(train_data,test_data,train_labels,test_labels) = dset.require_new_RNN(25,20)
train_data=np.array(train_data)
test_data=np.array(test_data)
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)

model = LinearSVC()
model.fit(train_data, train_labels)
prediction = model.predict(test_data)
acc_svm = accuracy_score(test_labels, prediction)
print ("Linear SVM accuracy: ",acc_svm)

# Accuracy : 35.47%
# Time of Execution : ~23m

/home/dradisavljevic/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Linear SVM accuracy:  0.425


In [25]:
# ANN Attempt Number 1

from keras.models import Sequential
from keras.layers.core import Activation, Dense
from keras.optimizers import SGD


# prepare model
model = Sequential()
model.add(Dense(70, input_dim=22500))
model.add(Activation('relu'))
model.add(Dense(140))
model.add(Activation('relu'))
model.add(Dense(74))
model.add(Activation('relu'))
# compile model with optimizer
sgd = SGD(lr=0.1, decay=0.001, momentum=0.7)
model.compile(loss='mean_squared_error', optimizer=sgd)

# training
train_data=np.array(train_data)
train_labels=np.array(train_labels)
nsamples, nx, ny, nz = train_data.shape
d2_train_data = train_data.reshape((nsamples,nx*ny*nz))
training = model.fit(d2_train_data, train_labels, nb_epoch=300, batch_size=222, verbose=0)

scores = model.evaluate(d2_train, train_labels, verbose=0)

# Accuracy : 1.93%
# Time of Execution : ~15s

ValueError: not enough values to unpack (expected 4, got 2)

In [15]:
# Multiple Layer Long Short Term Memory through usage of lasagne

import numpy as np
import time
import theano
import theano.tensor as T
import lasagne
import sklearn.metrics
import lasagne.layers as L

from lasagne.layers import InputLayer, LSTMLayer, ReshapeLayer, DenseLayer, GaussianNoiseLayer


# Number of Units in hidden layers
L1_UNITS = 50
L2_UNITS = 100

# Training Params 
LEARNING_RATE = 0.001
N_BATCH = 10
NUM_EPOCHS = 1500

# Load the dataset
print("Loading data...")
(train,test,train_labels,test_labels) = dset.require_new(8,2, True)

train=np.array(train)
test=np.array(test)
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)
train = train.reshape(train.shape[0], 150, 150).astype('float32')
test = test.reshape(test.shape[0], 150, 150).astype('float32')

num_feat    = train.shape[1]
num_classes = np.unique(test).size

# Generate sequence masks (redundant here)
mask_train = np.ones((train.shape[0], train.shape[1]))
mask_test  = np.ones((test.shape[0], test.shape[1]))


# Model
tanh = lasagne.nonlinearities.tanh
relu = lasagne.nonlinearities.rectify
soft = lasagne.nonlinearities.softmax

# Network Architecture
l_in = InputLayer(shape=(2, 3, num_feat))
batchsize, seqlen, _ = l_in.input_var.shape
print(num_classes)
print(batchsize)

l_noise = GaussianNoiseLayer(l_in, sigma=0.6) 
l_mask  = InputLayer(shape=(2, 3))

l_rnn_1 = LSTMLayer(l_noise, num_units=L1_UNITS, mask_input=l_mask)
l_in_drop = lasagne.layers.DropoutLayer(l_rnn_1, p=0.25)
l_rnn_2 = LSTMLayer(l_in_drop, num_units=L2_UNITS)
l_in_drop2 = lasagne.layers.DropoutLayer(l_rnn_2, p=0.1)
l_shp   = ReshapeLayer(l_in_drop2,(-1, L2_UNITS))
l_dense = DenseLayer(l_shp, num_units=num_classes, nonlinearity=soft)
l_out   = ReshapeLayer(l_dense, (2, 3, num_classes)) 
print( L.get_output(l_out))

# Symbols and Cost Function
target_values = T.ivector('target_output')

network_output   = L.get_output(l_out)
predicted_values = network_output[:, -1]
prediction = T.argmax(predicted_values, axis=1)
all_params = L.get_all_params(l_out, trainable=True)

cost = lasagne.objectives.categorical_crossentropy(predicted_values, target_values)
cost = cost.mean()



# Compute SGD updates for training
print("Computing updates ...")
updates = lasagne.updates.rmsprop(cost, all_params, LEARNING_RATE)

# Theano functions for training and computing cost
print("Compiling functions ...")
training   = theano.function(
    [l_in.input_var, target_values, l_mask.input_var], cost, updates=updates,allow_input_downcast=True)
predict = theano.function([l_in.input_var, l_mask.input_var], prediction,allow_input_downcast=True)
compute_cost = theano.function([l_in.input_var, target_values, l_mask.input_var], cost,allow_input_downcast=True)




# Training
print("Training ...")
num_batches_train = int(np.ceil(len(train) / N_BATCH))
train_losses = []
valid_losses = []

for epoch in range(NUM_EPOCHS):
    now = time.time
    losses = []        

    batch_shuffle = np.random.choice(train.shape[0], train.shape[0], False)
    sequences   = train[batch_shuffle]
    labels      = train_labels[batch_shuffle]
    train_masks = mask_train[batch_shuffle]

    for batch in range(num_batches_train):
        batch_slice = slice(N_BATCH * batch,
                            N_BATCH * (batch + 1))
        
        X_batch = sequences[batch_slice]
        y_batch = labels[batch_slice]
        m_batch = train_masks[batch_slice]

        loss = training(X_batch, y_batch, m_batch)
        losses.append(loss)
                
    train_loss = np.mean(losses)
    train_losses.append(train_loss)
    
    valid_loss = compute_cost(test, test_labels, mask_test)
    valid_losses.append(valid_loss)

    test_pred   = predict(test, mask_test)
    accuracy = sklearn.metrics.accuracy_score(test_labels, test_pred)


    print('Current epoch:', epoch+1,'|', 'Number of Epochs:', NUM_EPOCHS,'|','Train loss:', train_loss,'|','Validation loss:', valid_loss,'|','Accuracy:', accuracy)
    
    
# Accuracy : peaks at ~70%, starts at 1%
# Time of Execution : ~5h
# Time per Epoch : ~21s

Loading data...
8764
Subtensor{int64}.0


ValueError: expected an ndarray

In [32]:
#ANN attempt number 2

from keras.models import Sequential
from keras.layers.core import Dense
import os

folder='images50x50'

rel_path = 'images50x50'
size = 2500 # 50x50 slicica

folders = os.listdir(rel_path)

kana_dict = dataset.characters()
if os.path.isfile('dset50x50.hdf5'):
    os.remove('dset50x50.hdf5')
hiragana_dataset = dataset.HiraSet('dset50x50', size)

for folder in folders:
    entry = dataset.HiraEntry(folder, kana_dict[folder])

    files = os.listdir(rel_path + '/' + folder)
    for file in files:
        img = imread(rel_path + '/' + folder + '/' + file)
        re_img = np.reshape(img, size)
        flt_img = re_img / 65535.0
        print(flt_img)

        entry.add(flt_img)

    hiragana_dataset.add(entry)
dset50x50 = dataset.HiraSet('dset50x50', 2500)    
dset50x50.pull()


(train,test,train_labels,test_labels) = dset50x50.require_new(25,20)
train=np.array(train)
test=np.array(test)
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)



model = Sequential()
model.add(Dense(2500, input_dim=2500, kernel_initializer='normal', activation='relu'))
model.add(Dense(74, kernel_initializer='normal', activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# training
training = model.fit(train, train_labels, epochs=300, batch_size=100, verbose=0)
scores = model.evaluate(test, test_labels, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

# Accuracy : ~52.16%
# Time of Execution : ~6m

[0.5238117  0.51984436 0.51853208 ... 0.51682307 0.52912184 0.52379644]
[0.50218967 0.51218433 0.51113146 ... 0.51963073 0.52581064 0.51599908]
[0.53804837 0.53249409 0.53029679 ... 0.52802319 0.53154803 0.53080034]
[0.53214313 0.5335317  0.53224994 ... 0.5295491  0.52291142 0.53176165]
[0.50536355 0.5013962  0.51158923 ... 0.52001221 0.51343557 0.51914244]
[0.51874571 0.51924926 0.51209277 ... 0.5088426  0.51419852 0.51692988]
[0.50785077 0.50725566 0.51216907 ... 0.51213855 0.51491569 0.51113146]
[0.50905623 0.50585183 0.50952926 ... 0.51140612 0.51653315 0.51442741]
[0.5079118  0.51363394 0.52433051 ... 0.52018006 0.51934081 0.51386282]
[0.51627375 0.52449836 0.52239261 ... 0.51963073 0.52057679 0.52358282]
[0.52349126 0.52326238 0.52687877 ... 0.52616159 0.52149233 0.51741817]
[0.52951858 0.52874037 0.53948272 ... 0.52690929 0.52339971 0.52565805]
[0.51503777 0.51674678 0.51377127 ... 0.5205005  0.52491035 0.52265202]
[0.5198291  0.52303349 0.52845045 ... 0.52079042 0.52121767 0.49

[0.48035401 0.48033875 0.48099489 ... 0.48290227 0.49164569 0.49785611]
[0.48931106 0.49727626 0.49398032 ... 0.50118257 0.49825284 0.49997711]
[0.49439231 0.49430076 0.49169146 ... 0.51293202 0.50684367 0.50763714]
[0.47824826 0.47173266 0.46907759 ... 0.4789807  0.48502327 0.48703746]
[0.44806592 0.44274052 0.44028382 ... 0.45932708 0.46405737 0.46616312]
[0.44109255 0.44098573 0.44060426 ... 0.46305028 0.46253147 0.45818265]
[0.43231861 0.42903792 0.43270008 ... 0.45929656 0.4546273  0.45130083]
[0.44277104 0.45314717 0.44556344 ... 0.46424048 0.46398108 0.46898604]
[0.45072099 0.44704356 0.44379339 ... 0.46971847 0.47006943 0.46512551]
[0.44020752 0.43186084 0.44481575 ... 0.46668193 0.45946441 0.46109712]
[0.4459144  0.44522774 0.44545663 ... 0.46318761 0.46469825 0.46416419]
[0.43253223 0.43747616 0.44136721 ... 0.45766384 0.45970855 0.46570535]
[0.43802548 0.44335088 0.44101625 ... 0.459205   0.45908293 0.46842145]
[0.45256733 0.45439841 0.44972915 ... 0.46816205 0.46973373 0.46

[0.51287098 0.51082628 0.51340505 ... 0.52750439 0.53409628 0.52733654]
[0.51494621 0.51285573 0.52243839 ... 0.52890822 0.53075456 0.52581064]
[0.53365377 0.53383688 0.52927443 ... 0.53838407 0.54290074 0.54010834]
[0.51892882 0.52298772 0.52343023 ... 0.5313344  0.53060197 0.54058137]
[0.51198596 0.51740291 0.52080568 ... 0.53037308 0.52442206 0.52848096]
[0.51738766 0.51955444 0.52779431 ... 0.53694972 0.53893339 0.5326009 ]
[0.51157397 0.50955978 0.51952392 ... 0.53150225 0.5288777  0.52593271]
[0.53984894 0.5417258  0.53934539 ... 0.552224   0.54995041 0.5518883 ]
[0.53362325 0.52851148 0.52834363 ... 0.54834821 0.55225452 0.55132372]
[0.51258106 0.51343557 0.51847105 ... 0.52344549 0.53397421 0.53057145]
[0.51827268 0.51927977 0.52501717 ... 0.52912184 0.53179217 0.52925917]
[0.52080568 0.53302815 0.53299763 ... 0.54361791 0.54842451 0.54500649]
[0.53398947 0.53310445 0.53148699 ... 0.55271229 0.54999619 0.54949264]
[0.52814527 0.52655833 0.5226215  ... 0.54392309 0.54459449 0.54

[0.54491493 0.55211719 0.54497597 ... 0.53786526 0.54241245 0.53589685]
[0.52568856 0.52632944 0.5365835  ... 0.50429541 0.50550088 0.49976349]
[0.53728542 0.53835355 0.54740215 ... 0.54454871 0.54935531 0.5466392 ]
[0.51245899 0.51429007 0.51552605 ... 0.50595865 0.50699626 0.49962615]
[0.54386206 0.54699016 0.54738689 ... 0.53609522 0.53501183 0.54052033]
[0.51851682 0.51589227 0.5175555  ... 0.51143664 0.51212329 0.50899519]
[0.54120699 0.53896391 0.53856718 ... 0.53777371 0.53434043 0.53496605]
[0.53568322 0.53960479 0.54213779 ... 0.54322118 0.54737163 0.55397879]
[0.52137026 0.51511406 0.50928511 ... 0.49784085 0.50402075 0.50702678]
[0.50577554 0.51090257 0.50926986 ... 0.49552148 0.49874113 0.50350195]
[0.53997101 0.53813993 0.54116121 ... 0.54148165 0.54515908 0.54653239]
[0.54848554 0.5407187  0.54926375 ... 0.54831769 0.54921798 0.54608988]
[0.52747387 0.51717403 0.51827268 ... 0.49799344 0.50284581 0.50467689]
[0.51960021 0.51903563 0.52298772 ... 0.50466163 0.49996185 0.50

[0.45593957 0.45831998 0.45456626 ... 0.47594415 0.47496757 0.47067979]
[0.44638743 0.44246586 0.44200809 ... 0.47887388 0.48000305 0.48349737]
[0.44824903 0.44454108 0.44556344 ... 0.47043565 0.47417411 0.46649882]
[0.45363546 0.45174334 0.45412375 ... 0.47301442 0.47571527 0.47582208]
[0.4646067  0.46091402 0.46538491 ... 0.47724117 0.47841611 0.47866026]
[0.44306096 0.44536507 0.44795911 ... 0.46497292 0.46236362 0.45871672]
[0.46285191 0.46480507 0.46387427 ... 0.47629511 0.47791257 0.47661555]
[0.45088884 0.45009537 0.44208438 ... 0.46614786 0.46942855 0.47206836]
[0.44783703 0.44759289 0.45061418 ... 0.46210422 0.46053254 0.46543069]
[0.45243    0.45305562 0.44628061 ... 0.47573053 0.46718547 0.46306554]
[0.45465782 0.45606165 0.45702296 ... 0.47658503 0.47989624 0.48436713]
[0.45931182 0.4562295  0.45874723 ... 0.47641718 0.46845197 0.47461662]
[0.44875257 0.45598535 0.45505455 ... 0.47512016 0.47951476 0.47684443]
[0.45282673 0.45439841 0.45230793 ... 0.46964218 0.47338064 0.47

...

[0.48790723 0.47154955 0.46826886 ... 0.46709392 0.45838102 0.45545129]
[0.45951019 0.45619898 0.46111238 ... 0.45412375 0.44475471 0.44563973]
[0.45139239 0.44992752 0.44020752 ... 0.45937285 0.45630579 0.45491722]
[0.48033875 0.47959106 0.48053712 ... 0.48471809 0.485832   0.48926528]
[0.48903639 0.49343099 0.49373617 ... 0.48967727 0.49332418 0.49489586]
[0.49863432 0.49506371 0.4920119  ... 0.48992142 0.48671702 0.4888838 ]
[0.48459602 0.48546578 0.48587778 ... 0.48798352 0.48722057 0.48331426]
[0.49620813 0.5054551  0.50109102 ... 0.50222019 0.50473793 0.5029221 ]
[0.48693065 0.48526741 0.48844129 ... 0.48818189 0.49478904 0.49822232]
[0.48786145 0.49066911 0.48758679 ... 0.48685435 0.49405661 0.49628443]
[0.48284123 0.47689021 0.47598993 ... 0.49051652 0.48331426 0.48729686]
[0.49298848 0.48361944 0.48130007 ... 0.48824292 0.48603037 0.48610666]
[0.49918364 0.49564355 0.48807507 ... 0.50022126 0.49721523 0.49892424]
[0.49677272 0.49637598 0.50405127 ... 0.51442741 0.50463111 0.49

[0.52933547 0.5276875  0.52730602 ... 0.52188907 0.52480354 0.52216373]
[0.53417258 0.53160906 0.53730068 ... 0.5238117  0.52271305 0.52133974]
[0.52538338 0.5254139  0.52384222 ... 0.51570916 0.5200885  0.51453422]
[0.53202106 0.52936599 0.53147173 ... 0.52382696 0.52451362 0.51810483]
[0.53829252 0.5410544  0.53681239 ... 0.51717403 0.51812009 0.51178759]
[0.42105745 0.41591516 0.42014191 ... 0.45427634 0.45685512 0.45243   ]
[0.41747158 0.4133669  0.41510643 ... 0.45865568 0.46053254 0.45159075]
[0.42522316 0.43071641 0.43476005 ... 0.4635996  0.45899138 0.45626001]
[0.41484703 0.41701381 0.41751736 ... 0.44679942 0.4532845  0.46082246]
[0.4318761  0.42995346 0.43150988 ... 0.4711223  0.47936217 0.47530327]
[0.41324483 0.42301061 0.42412451 ... 0.45587854 0.45680934 0.46247044]
[0.42311742 0.42276646 0.41983673 ... 0.45243    0.45493248 0.46166171]
[0.41853971 0.41898222 0.42323949 ... 0.44919509 0.4461738  0.4449073 ]
[0.41789883 0.42365148 0.42403296 ... 0.45542077 0.45610742 0.45

[0.53977264 0.53357748 0.54087129 ... 0.53299763 0.53275349 0.54093233]
[0.55388724 0.54776837 0.54233616 ... 0.53795682 0.53675135 0.54512856]
[0.53519493 0.54200046 0.5491722  ... 0.5534905  0.54807355 0.54245823]
[0.53481346 0.53586633 0.54583047 ... 0.52635996 0.52623789 0.53450828]
[0.53333333 0.53916228 0.53975738 ... 0.54329747 0.54502174 0.54460975]
[0.54668498 0.55649653 0.53963531 ... 0.54613565 0.5354696  0.53813993]
[0.53693446 0.53621729 0.52932021 ... 0.54421302 0.54157321 0.54148165]
[0.52907607 0.52390326 0.52747387 ... 0.53807889 0.54152743 0.5364004 ]
[0.53516442 0.5347219  0.53737697 ... 0.54239719 0.54100862 0.54195468]
[0.53022049 0.53363851 0.53269245 ... 0.53292134 0.53728542 0.540383  ]
[0.53095293 0.53464561 0.53492027 ... 0.5372549  0.5376669  0.53754482]
[0.5372549  0.53896391 0.53397421 ... 0.54311437 0.53650721 0.53490501]
[0.53434043 0.53273823 0.53115129 ... 0.53730068 0.54230564 0.55022507]
[0.55269703 0.55019455 0.55732052 ... 0.564477   0.55922789 0.55

[0.53633936 0.53853666 0.53910124 ... 0.52677195 0.52893874 0.53777371]
[0.48764782 0.48885328 0.49761196 ... 0.48364996 0.48178836 0.47960632]
[0.5372549  0.54024567 0.54314488 ... 0.53041886 0.53731594 0.53871977]
[0.5385214  0.53510338 0.535729   ... 0.53373007 0.52686351 0.52498665]
[0.49227131 0.49069963 0.49370565 ... 0.47467765 0.47775998 0.48029297]
[0.53849088 0.5394522  0.53940642 ... 0.53830777 0.53279927 0.53791104]
[0.49811551 0.49585718 0.49150835 ... 0.48509957 0.48155947 0.48999771]
[0.52928969 0.53113603 0.52755016 ... 0.51892882 0.52394903 0.52959487]
[0.48830396 0.49831388 0.50414282 ... 0.48838025 0.49059281 0.49672694]
[0.54375525 0.53655299 0.53170062 ... 0.53389792 0.5385977  0.53359274]
[0.50148775 0.49831388 0.49120317 ... 0.49762722 0.48987564 0.49404135]
[0.53986419 0.54338903 0.54804303 ... 0.54448768 0.54279393 0.54413672]
[0.52512398 0.52263676 0.52791638 ... 0.52866407 0.52800793 0.53649195]
[0.55152209 0.54348058 0.5463035  ... 0.55060655 0.5485008  0.55

KeyboardInterrupt: 

In [ ]:
# VGG16 neural network with pretrained weights. Not Tested as it works with coloured images.

from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv.summary()

#Create your own input format (here 3x200x200)
input = Input(shape=(3,150,150),name = 'image_input')

#Use the generated model
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(74, activation='softmax', name='predictions')(x)

#Create your own model
my_model = Model(input=input, output=x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()


#Then training with your data !